In [1]:
%load_ext autoreload
import os,torch,torch.nn as nn
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import NeighborLoader
from VeloFusion.util import EarlyStopping
import comet_ml
from comet_ml import Experiment

from VeloFusion.model import VeloFusion

import dynamo as dyn
import numpy as np
import pandas as pd

# 导入数据
abm1 =dyn.read_h5ad("/home/csj/mywork/VeloFusion_v1/datasets/ILC/221121-abm1.dynamo.h5ad")
abm2 =dyn.read_h5ad("/home/csj/mywork/VeloFusion_v1/datasets/ILC/221121-abm2.dynamo.h5ad")
#abm_joint = abm1.concatenate(abm2)

goi1 = abm1.var[abm1.var['use_for_transition']].index.tolist()
goi2 = abm2.var[abm2.var['use_for_transition']].index.tolist()
goi  = np.unique(goi1+goi2)
del goi1,goi2

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

goi = intersection(goi, abm1.var_names)
goi = intersection(goi, abm2.var_names)
print(len(goi), "genes selected")

# 构图
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu') #modify

G1 = torch.load('datasets/ILC/ABM1.graph.pt',map_location=device)
G1.n_id = torch.arange(G1.num_nodes)
G1.cond = torch.tensor(0).repeat(G1.num_nodes) 

G2 = torch.load('datasets/ILC/ABM2.graph.pt',map_location=device)
G2.n_id = torch.arange(G2.num_nodes)
G2.cond = torch.tensor(1).repeat(G2.num_nodes)

def concat_graphs(graph_list):
    new_x = []
    for g in graph_list:
        new_x.append(g.x)
    new_x = torch.cat(new_x, 0)
    
    index_increment = [0]
    for g in graph_list[:-1]:
        index_increment.append(g.num_nodes)
        
    new_edge_index = []
    new_n_id = []
    new_cond = []
    for g_idx in range(len(graph_list)):
        g = graph_list[g_idx]
        new_edge_index.append(g.edge_index + index_increment[g_idx])
        new_n_id.append(g.n_id)
        new_cond.append(g.cond)
    new_edge_index = torch.cat(new_edge_index, dim =1)
    new_n_id = torch.cat(new_n_id, dim=0)
    new_cond = torch.cat(new_cond, dim=0)
        
    new_edge_weight = torch.tensor([], device=graph_list[0].edge_weight.device) 
    for g in graph_list:
        new_edge_weight = torch.cat([new_edge_weight, g.edge_weight], dim=0)
        
    G_joint= Data(x=new_x , edge_index=new_edge_index, edge_weight=new_edge_weight)
    G_joint.n_id = new_n_id
    G_joint.cond = new_cond
    return G_joint

Gj= concat_graphs([G1,G2])

# 搭建模型
hyper_params = {
    "lr": 1e-4,
    "use_lr_scheduler":True,
    "weight_decay": 1e-2,
    "nEpoch": 10000,
    "early_stop_after":300,
    "batch_size": 1,
    "FC_layer_sizes" :[100, 800],
    "num_neighbors":[None],
    "neighbor_loader_bs":None,
    "warmup_epoch": 20,
    "use_early_stop":True,
    "num_features": G1.num_features,
    "z_dim" : 50,
    "residual_encoder" :False, # modify
    "gravity_lambda" : 0.5,
    "KLD_beta": 100, #modify
    "use_negative_balancing" :True,
    "feature_gamma1" : 200,
    "edge_gamma2" :100,
    "mmd_on" : 'none',
    "mmd_beta": 1,
    "mmd_weight":100,
    "feature_recon_w_v":0.7,
    "checkpt_path":'./checkpoint/velofusion.RGCN.model_V4.3_z50FC2Beta100NoRes.pt' # modify
}
checkpt_path = hyper_params['checkpt_path']

# Data 
from torch_geometric.data import Batch
batch_data = Batch.from_data_list([G1, G2]).to(device)
train_loader = [batch_data]*20

# Model 
num_features = G1.num_features
model = VeloFusion(feature_dim= num_features, hidden_dim=400, 
                   FC_layer_sizes = hyper_params['FC_layer_sizes'],
                   z_dim=hyper_params['z_dim'], cond_dim=2,
                   residual_encoder=hyper_params['residual_encoder'],
                   gravity_lambda=hyper_params['gravity_lambda'], 
                   KLD_beta=hyper_params['KLD_beta'], 
                   use_negative_balancing=hyper_params['use_negative_balancing'], 
                   feature_gamma1=hyper_params['feature_gamma1'], 
                   edge_gamma2=hyper_params['edge_gamma2'], 
                   mmd_on =hyper_params['mmd_on'], 
                   mmd_beta=hyper_params['mmd_beta'], 
                   mmd_weight=hyper_params['mmd_weight'],
                   feature_recon_w_v=hyper_params['feature_recon_w_v']
                  )
model = model.to(device)

# Optimization
from lion_pytorch import Lion
optimizer = Lion(model.parameters(), 
                 lr=hyper_params['lr'], 
                 weight_decay=hyper_params['weight_decay'])

from torch.optim.lr_scheduler import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.75, patience=10, cooldown=30)


# experiment info
use_comet =True
experiment = Experiment(
    api_key="tgisrDhwWNlSvYukC8oEkH4sT",
    project_name="velofusion",
    workspace="chansigit",
    auto_log_co2=False,
    disabled = False
)
experiment.log_parameters(hyper_params)

# train
from tqdm.autonotebook import tqdm
from collections import OrderedDict

model.train()
nEpoch = hyper_params['nEpoch']
use_early_stop = hyper_params['use_early_stop']
use_lr_scheduler = hyper_params['use_lr_scheduler']
early_stop_after = hyper_params['early_stop_after']
pbar = tqdm(total=nEpoch*len(train_loader))

total_step=0
early_stopping = EarlyStopping(patience=20, verbose=True, path=checkpt_path)
for epoch in range(nEpoch):
    optimizer.zero_grad()
    if use_lr_scheduler and (epoch<hyper_params['warmup_epoch']):
        warmup=(epoch+1)/hyper_params['warmup_epoch']*(hyper_params['lr']-1e-7)
        optimizer.param_groups[0]['lr'] = warmup
        
    epoch_total_loss = epoch_size = 0
    batch_id = 0
    for sub in train_loader:
        sub.to(device)
        loss, loss_info = model.loss(x=sub.x, 
                                     edge_index=sub.edge_index, 
                                     edge_weight=sub.edge_weight, 
                                     cond_num=sub.cond)
        loss.backward()
        optimizer.step()
        
        epoch_total_loss += float(loss) * 1 
        epoch_size += 1
        batch_id +=1
        total_step +=1
        pbar.update(1)
        if use_comet:
            experiment.log_metric("loss", loss.cpu().item(), epoch=epoch, step=total_step)
            experiment.log_metric("FRL", loss_info['FRL'].cpu().item(), epoch=epoch, step=total_step)
            experiment.log_metric("FRL_x", loss_info['FRL_x'].cpu().item(), epoch=epoch, step=total_step)
            experiment.log_metric("FRL_v", loss_info['FRL_v'].cpu().item(), epoch=epoch, step=total_step)
            experiment.log_metric("ERL", loss_info['ERL'].cpu().item(), epoch=epoch, step=total_step)
            experiment.log_metric("KLD", loss_info['KLD'].cpu().item(), epoch=epoch, step=total_step)
            experiment.log_metric("MMD", loss_info['MMD'].cpu().item(), epoch=epoch, step=total_step)
            experiment.log_metric("lr", optimizer.param_groups[0]['lr'], epoch=epoch, step=total_step)

        if batch_id % 20 == 0:
            pbar.set_postfix(OrderedDict(
                batch=batch_id, loss=loss.cpu().item(),
                FRL=loss_info['FRL'].cpu().item(), ERL=loss_info['ERL'].cpu().item(),
                KLD=loss_info['KLD'].cpu().item(), MMD=loss_info['MMD'].cpu().item(),
                lr=optimizer.param_groups[0]['lr'])
            )
            
    epoch_avg_loss = epoch_total_loss / epoch_size
    pbar.set_description("[Epoch %d avg_loss=%.2f]" % (epoch,epoch_avg_loss))
    
    if use_lr_scheduler:
        lr_scheduler.step(epoch_avg_loss)
        
    if use_early_stop and (epoch > early_stop_after) :
        early_stopping(epoch_avg_loss, model)
        if early_stopping.early_stop:
            print("Early stopping!")
            break

experiment.end()

torch.save(model, checkpt_path) 

|-----> setting visualization default mode in dynamo. Your customized matplotlib settings might be overritten.


/home/csj/miniconda3/envs/celltensor/lib/python3.8/site-packages/nxviz/__init__.py:18: UserWarning: 
nxviz has a new API! Version 0.7.3 onwards, the old class-based API is being
deprecated in favour of a new API focused on advancing a grammar of network
graphics. If your plotting code depends on the old API, please consider
pinning nxviz at version 0.7.3, as the new API will break your old code.

To check out the new API, please head over to the docs at
https://ericmjl.github.io/nxviz/ to learn more. We hope you enjoy using it!

(This deprecation message will go away in version 1.0.)

  warnings.warn(


1451 genes selected


COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/home/csj/mywork/VeloFusion_v1' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_GIT_DIRECTORY`
COMET INFO: Experiment is live on comet.ml https://www.comet.com/chansigit/velofusion/4e0dddb74b2d4612b9580ae8fb63a4df



  0%|          | 0/200000 [00:00<?, ?it/s]

...

EarlyStopping counter: 5 out of 20
Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...
EarlyStopping counter: 1 out of 20
EarlyStopping counter: 2 out of 20
EarlyStopping counter: 3 out of 20
EarlyStopping counter: 4 out of 20
EarlyStopping counter: 5 out of 20
Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...
EarlyStopping counter: 1 out of 20
EarlyStopping counter: 2 out of 20
Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...
Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...
EarlyStopping counter: 1 out of 20
Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...
EarlyStopping counter: 1 out of 20
Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...
EarlyStopping counter: 1 out of 20
EarlyStopping counter: 2 out of 20
EarlyStopping counter: 3 out of 20
Valid

EarlyStopping counter: 9 out of 20
EarlyStopping counter: 10 out of 20
EarlyStopping counter: 11 out of 20
EarlyStopping counter: 12 out of 20
EarlyStopping counter: 13 out of 20
EarlyStopping counter: 14 out of 20
EarlyStopping counter: 15 out of 20
EarlyStopping counter: 16 out of 20
EarlyStopping counter: 17 out of 20
EarlyStopping counter: 18 out of 20
EarlyStopping counter: 19 out of 20
Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...
Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...
Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...
EarlyStopping counter: 1 out of 20
Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...
EarlyStopping counter: 1 out of 20
EarlyStopping counter: 2 out of 20
EarlyStopping counter: 3 out of 20
Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...
Validation loss

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/chansigit/velofusion/4e0dddb74b2d4612b9580ae8fb63a4df
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     ERL [61400]   : (0.03824084997177124, 1.2515509128570557)
COMET INFO:     FRL [61400]   : (0.06653119623661041, 0.25720977783203125)
COMET INFO:     FRL_v [61400] : (0.03783378377556801, 0.2029101848602295)
COMET INFO:     FRL_x [61400] : (0.13335338234901428, 0.38390883803367615)
COMET INFO:     KLD [61400]   : (3.6153938176575195e-10, 0.009188627824187279)
COMET INFO:     MMD           : 0.0
COMET INFO:     loss [61400]  : (17.713863372802734, 175.64041137695312)
COMET INFO:     lr [61400]    : (4.9950000000000005e-06, 9.99e-05)
COMET INFO:   Parameters:
COMET INFO:     FC_layer_sizes         : [100, 800]
COMET INFO:     KLD_beta     

EarlyStopping counter: 20 out of 20
Early stopping!


COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET INFO: Uploading metrics, params, and assets to Comet before program termination (may take several seconds)
COMET INFO: The Python SDK has 3600 seconds to finish before aborting...
COMET WARNING: Websocket connection didn't closed properly after 30 seconds
